In [1]:
# First we want to check what version of python we are running here: 
import sys
print("Python version: " + str(sys.version_info.major) + "." + str(sys.version_info.minor) + "." + str(sys.version_info.micro))

Python version: 3.8.3


In [2]:
!pip install torch
!pip install torchtext

In [1]:
import torchtext
print("Successfully imported TorchText module from PyTorch.")

Successfully imported TorchText module from PyTorch.


In [3]:
# Import some datasets from TorchText
from torchtext.datasets import text_classification
print("Successfully imported some datasets from TorchText.")

Successfully imported some datasets from TorchText.


In [4]:
# Creating a folder called 'data'
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
    
# Specifying how to break up the dataset into n-words 
NGRAMS = 2

# Load the training dataset and testing dataset
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)

.data\ag_news_csv.tar.gz: 11.8MB [00:02, 5.04MB/s]
120000lines [00:31, 3816.44lines/s]
120000lines [01:05, 1831.94lines/s]
7600lines [00:04, 1680.21lines/s]


In [6]:
# Import the neural network package from PyTorch 
import torch.nn as nn

# We also want to the functional module which contains all the functions in the torch.nn library.
import torch.nn.functional as F

# Creating a class called TextSentiment that takes in the neural network module from Pytorch.
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        # The EmbeddingBag module finds the sum or the mean of all the embeddings.
        # An embedding allows you to map low-dimensional real vectors that can represent each words to other words that are similar. 
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        # This does a Linear Transformation which allows the model to learn the weights between the embeddings and maps it to an output class
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        # intialization range set at 0.5
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    # This is the forward propagation which essentially feeds input data into each layer of the model 
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [7]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUM_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS)

print("The vocabulary size is: ", VOCAB_SIZE)
print("The number of classes is: ", NUM_CLASS)

The vocabulary size is:  1308844
The number of classes is:  4


In [1]:
def generate_batch(batch):
    # A tensor is used to represent n-Dimensions of features. It looks like a matrix but it's not a matrix, a matrix is simply used to visualize a tensor. 
    # The first part of the entry is the class label (e.g. what type of news article).
    label = torch.tensor([entry[0] for entry in batch])
    # The second part of the entry is the text. 
    text = [entry[1] for entry in batch]
    # These are the offset values in storage which indicates where the tensors start from.
    offsets = [0] + [len(entry) for entry in text]
    
    # torch.Tensor.cumsum returns a cumulative sum of all the elements in the dimension.
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    
    # torch.cat concatenates whatever you give it together. 
    text = torch.cat(text)
    return text, offsets, label

In [ ]:
from torch.utils.data import DataLoader

#Dataloder is used to load the data and then send it to the model for traning and validation 
def train_func(sub_train_):

    # Train the model 
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    #  a function dataloder is called here with the parameters to be passed for thr validation of our model
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate  fir the number of epoches taken
    scheduler.step()
    
    return train_loss / len(sub_train_), train_acc / len(sub_train_)
    
#testing the model
def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
#Evaluating the data with the test dataset..
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
#finally the locc and the accuracy are calculated
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

In [ ]:
#Evaluating the model on some random data 
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

#different labels taken into consideration
ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

#Use of the trained model and the bgrams for the classification
def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1
#example input text taken
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")
#final classification of the taken input text
print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])